In [ ]:
import os
import pandas as pd
import time
from google.cloud import bigquery
from google.cloud import automl_v1beta1 as automl

# --- Configurações do Projeto e Dados ---
# Substitua estas variáveis pelos valores do seu projeto Google Cloud
PROJECT_ID = "seu-project-id"  # Ex: "my-gcp-project-12345"
REGION = "us-central1"        # Ex: "us-central1", "eu-west4", "southamerica-east1"
                              # Escolha uma região onde o AutoML Tables esteja disponível.
                              # Ref: https://cloud.google.com/automl/docs/locations

# Caminho do seu arquivo CSV
CSV_FILE_PATH = '..\\Bases\\Originais\\pns2019.csv' # Seu caminho original

# --- Configurações do BigQuery ---
BIGQUERY_PROJECT_ID = PROJECT_ID
BIGQUERY_DATASET_ID = "pns_data"           # Um novo ou existente dataset do BigQuery para seus dados
BIGQUERY_TABLE_ID = "pns2019_raw"          # Nome da tabela no BigQuery onde o DataFrame será salvo

# --- Configurações do AutoML Tables (para depois de carregar para o BigQuery) ---
DATASET_DISPLAY_NAME = "pns2019_automl_dataset" # Nome visível para o seu dataset no AutoML
TARGET_COLUMN = "COLUNA_ALVO_DA_PNS"       # <--- **IMPORTANTE:** Mude para o nome exato da sua coluna alvo na PNS
                                           # Ex: Se você quer prever 'V0018' (idade), TARGET_COLUMN = 'V0018'

# Configurações do Treinamento do Modelo
MODEL_DISPLAY_NAME = f"{DATASET_DISPLAY_NAME}_modelo"
TRAIN_BUDGET_MILLI_NODE_HOURS = 1000       # 1 hora de treinamento (para testes)
OPTIMIZATION_OBJECTIVE = "MINIMIZE_RMSE"   # Ex: 'MINIMIZE_RMSE' para regressão (se TARGET_COLUMN for numérica contínua)
                                           # Ex: 'MAXIMIZE_AU_PRC' para classificação binária
                                           # Ex: 'MAXIMIZE_ACCURACY' para classificação multiclasse

# --- PARTE A: Carregar DataFrame para o BigQuery ---
def load_dataframe_to_bigquery(df: pd.DataFrame, project_id: str, dataset_id: str, table_id: str):
    """
    Carrega um DataFrame Pandas para uma tabela no Google BigQuery.
    Cria o dataset e a tabela se não existirem.
    """
    print(f"\n--- Parte A: Carregando DataFrame para o BigQuery ---")
    client = bigquery.Client(project=project_id)

    # Constrói a referência completa do dataset e da tabela
    dataset_ref = client.dataset(dataset_id)
    table_ref = dataset_ref.table(table_id)

    # 1. Verificar/Criar Dataset no BigQuery
    try:
        client.get_dataset(dataset_ref)
        print(f"Dataset BigQuery '{dataset_id}' já existe.")
    except Exception:
        print(f"Criando dataset BigQuery '{dataset_id}'...")
        dataset = bigquery.Dataset(dataset_ref)
        dataset.location = REGION # A localização do dataset deve ser a mesma da sua região AutoML
        client.create_dataset(dataset)
        print(f"Dataset BigQuery '{dataset_id}' criado.")

    # 2. Carregar DataFrame para a Tabela
    print(f"Carregando DataFrame para a tabela BigQuery '{table_id}'...")
    job_config = bigquery.LoadJobConfig(
        write_disposition="WRITE_TRUNCATE",  # Substitui a tabela se ela já existir
        # Se você tiver que forçar tipos, pode usar um schema:
        # schema=[
        #     bigquery.SchemaField("nome_coluna", "STRING"),
        #     bigquery.SchemaField("idade", "INTEGER"),
        # ]
    )

    try:
        job = client.load_table_from_dataframe(
            df, table_ref, job_config=job_config
        )
        job.result()  # Espera o job completar
        print(f"DataFrame carregado com sucesso para a tabela '{table_id}' no BigQuery.")
        print(f"Total de linhas carregadas: {job.output_rows}")
        return True
    except Exception as e:
        print(f"Erro ao carregar DataFrame para o BigQuery: {e}")
        return False

# --- PARTE B: Usar AutoML Tables com os Dados do BigQuery ---
def create_and_train_automl_tables_model(
    project_id: str,
    region: str,
    dataset_display_name: str,
    bigquery_source_uri: str,
    target_column: str,
    model_display_name: str,
    train_budget_milli_node_hours: int,
    optimization_objective: str
):
    """
    Cria um conjunto de dados no AutoML Tables, importa dados do BigQuery,
    define a coluna alvo e inicia o treinamento de um modelo.
    """
    print(f"\n--- Parte B: Treinando Modelo AutoML Tables ---")
    try:
        client = automl.TablesClient(project=project_id, region=region)
        print(f"Cliente AutoML Tables inicializado para o projeto '{project_id}' na região '{region}'.")

        # 1. Criar um novo conjunto de dados (Dataset) no AutoML Tables
        print(f"\n1. Criando conjunto de dados '{dataset_display_name}'...")
        dataset = {
            "display_name": dataset_display_name,
            "tables_dataset_metadata": {},
        }
        create_dataset_operation = client.create_dataset(
            parent=f"projects/{project_id}/locations/{region}",
            dataset=dataset
        )
        new_dataset = create_dataset_operation.result()
        dataset_name = new_dataset.name # Formato: projects/ID/locations/REGION/datasets/ID_DATASET
        print(f"   Conjunto de dados criado: {new_dataset.display_name} (ID: {dataset_name.split('/')[-1]})")

        # 2. Importar dados do BigQuery para o conjunto de dados
        print(f"\n2. Importando dados de '{bigquery_source_uri}' para o conjunto de dados '{dataset_display_name}'...")
        input_config = {
            "bigquery_source": {
                "uri": bigquery_source_uri
            }
        }
        import_data_operation = client.import_data(
            name=dataset_name,
            input_config=input_config
        )
        print("   Aguardando a importação dos dados... Isso pode levar alguns minutos.")
        import_data_operation.result() # Bloqueia até a importação ser concluída
        print("   Dados importados com sucesso para o AutoML Tables!")

        # 3. Obter as especificações das colunas e definir a coluna alvo
        print(f"\n3. Definindo a coluna alvo: '{target_column}'...")
        list_column_specs_response = client.list_column_specs(parent=dataset_name)
        column_specs = {s.display_name: s for s in list_column_specs_response}

        if target_column not in column_specs:
            raise ValueError(
                f"Erro: Coluna alvo '{target_column}' não encontrada no dataset do AutoML. "
                f"Colunas disponíveis: {list(column_specs.keys())}"
            )

        target_column_spec = column_specs[target_column]
        target_column_id = target_column_spec.name.split('/')[-1]

        # Atualiza o dataset para definir a coluna alvo
        update_dataset_dict = {
            "name": dataset_name,
            "tables_dataset_metadata": {
                "target_column_spec_id": target_column_id
            }
        }
        client.update_dataset(dataset=update_dataset_dict)
        print(f"   Coluna alvo '{target_column}' definida para o conjunto de dados.")

        # 4. Criar e Treinar o Modelo
        print(f"\n4. Iniciando o treinamento do modelo '{model_display_name}'...")
        model = {
            "display_name": model_display_name,
            "dataset_id": dataset_name.split('/')[-1],
            "tables_model_metadata": {
                "target_column_spec_id": target_column_id,
                "train_budget_milli_node_hours": train_budget_milli_node_hours,
                "optimization_objective": optimization_objective,
            }
        }

        create_model_operation = client.create_model(
            parent=f"projects/{project_id}/locations/{region}",
            model=model
        )
        print(f"   Operação de criação/treinamento do modelo iniciada: {create_model_operation.operation.name}")
        print(f"   O treinamento pode levar várias horas (orçamento de {train_budget_milli_node_hours/1000} horas).")
        print("   Você pode monitorar o progresso no console do Google Cloud > AutoML > Tables > Models.")
        print("   Aguarde a conclusão ou verifique o console para obter o modelo treinado.")

    except Exception as e:
        print(f"\nOcorreu um erro na parte do AutoML: {e}")
        print("Verifique suas configurações de projeto, IDs, nomes, região e autenticação.")
        print("Certifique-se de que as APIs necessárias estão habilitadas e as permissões corretas foram concedidas.")


# --- Execução Principal do Script ---
if __name__ == "__main__":
    # Verifica a autenticação antes de iniciar
    if "GOOGLE_APPLICATION_CREDENTIALS" not in os.environ:
        print("ERRO: A variável de ambiente 'GOOGLE_APPLICATION_CREDENTIALS' não está definida.")
        print("Por favor, defina-a com o caminho para sua chave JSON da conta de serviço.")
        print("Exemplo: export GOOGLE_APPLICATION_CREDENTIALS=\"/caminho/para/sua/chave.json\"")
        exit() # Sai do script se a autenticação não estiver configurada

    print("Variável de ambiente 'GOOGLE_APPLICATION_CREDENTIALS' detectada. Prosseguindo...")

    # 0. Carregar o DataFrame
    print(f"Lendo o arquivo CSV: {CSV_FILE_PATH}")
    try:
        df = pd.read_csv(CSV_FILE_PATH)
        print(f"DataFrame lido com sucesso. Formato: {df.shape}")
        # print("Primeiras 5 linhas do DataFrame:")
        # print(df.head())
        # print("Colunas disponíveis:", df.columns.tolist())

        # --- VALIDAR COLUNA ALVO ---
        if TARGET_COLUMN not in df.columns:
            print(f"\nERRO: A coluna alvo '{TARGET_COLUMN}' não foi encontrada no seu DataFrame.")
            print("Por favor, verifique o nome da coluna e atualize a variável `TARGET_COLUMN` no script.")
            print("Colunas presentes no CSV:", df.columns.tolist())
            exit()
        else:
            print(f"Coluna alvo '{TARGET_COLUMN}' encontrada no DataFrame.")

    except FileNotFoundError:
        print(f"ERRO: Arquivo CSV não encontrado no caminho: {CSV_FILE_PATH}")
        print("Por favor, verifique o caminho e o nome do arquivo.")
        exit()
    except Exception as e:
        print(f"ERRO ao ler o CSV: {e}")
        exit()

    # Construa a URI completa da sua tabela BigQuery para o AutoML usar
    BIGQUERY_SOURCE_URI = (
        f"bq://{BIGQUERY_PROJECT_ID}.{BIGQUERY_DATASET_ID}.{BIGQUERY_TABLE_ID}"
    )

    # --- Executar a Parte A: Carregar para o BigQuery ---
    if load_dataframe_to_bigquery(df, BIGQUERY_PROJECT_ID, BIGQUERY_DATASET_ID, BIGQUERY_TABLE_ID):
        # --- Executar a Parte B: Treinar o Modelo AutoML ---
        create_and_train_automl_tables_model(
            project_id=PROJECT_ID,
            region=REGION,
            dataset_display_name=DATASET_DISPLAY_NAME,
            bigquery_source_uri=BIGQUERY_SOURCE_URI,
            target_column=TARGET_COLUMN,
            model_display_name=MODEL_DISPLAY_NAME,
            train_budget_milli_node_hours=TRAIN_BUDGET_MILLI_NODE_HOURS,
            optimization_objective=OPTIMIZATION_OBJECTIVE
        )
    else:
        print("\nNão foi possível prosseguir com o treinamento do AutoML devido a erros no carregamento para o BigQuery.")

ERRO: A variável de ambiente 'GOOGLE_APPLICATION_CREDENTIALS' não está definida.
Por favor, defina-a com o caminho para sua chave JSON da conta de serviço.
Exemplo: export GOOGLE_APPLICATION_CREDENTIALS="/caminho/para/sua/chave.json"
Variável de ambiente 'GOOGLE_APPLICATION_CREDENTIALS' detectada. Prosseguindo...
Lendo o arquivo CSV: ..\Bases\Originais\pns2019.csv
DataFrame lido com sucesso. Formato: (293726, 1087)

ERRO: A coluna alvo 'COLUNA_ALVO_DA_PNS' não foi encontrada no seu DataFrame.
Por favor, verifique o nome da coluna e atualize a variável `TARGET_COLUMN` no script.
Colunas presentes no CSV: ['V0001', 'V0024', 'UPA_PNS', 'V0006_PNS', 'V0015', 'V0020', 'V0022', 'V0026', 'V0031', 'V0025A', 'V0025B', 'A001', 'A002010', 'A003010', 'A004010', 'A01001', 'A011', 'A005010', 'A005012', 'A00601', 'A009010', 'A01401', 'A01402', 'A01403', 'A01501', 'A016010', 'A018011', 'A018012', 'A018013', 'A018014', 'A018015', 'A018016', 'A018017', 'A018018', 'A018019', 'A018020', 'A018021', 'A01802

DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.

: 